In [ ]:
%matplotlib qt
import matplotlib.pyplot as plt
import pandas as pd
import geopandas as gpd
from pyproj import CRS
from geopy.geocoders import Nominatim

In [ ]:
gdf = gpd.read_file('/home/welling/geo/USA/all/tl_2021_us_county.shp')

In [ ]:
gdf.columns

In [ ]:
gdf = gdf[gdf.STATEFP=='06']  # select CA

In [ ]:
gdf.head()

In [ ]:
def add_area_and_label_coords(some_gdf):
    some_gdf['area'] = some_gdf.area
    some_gdf['coords'] = some_gdf['geometry'].apply(lambda x: x.representative_point().coords[:])
    some_gdf['coords'] = [coords[0] for coords in some_gdf['coords']]

In [ ]:
def plot_with_labels(some_gdf, ax, name_col=None, field_col=None, **kwargs):
    if field_col is None:
        some_gdf.plot(ax=ax, **kwargs)
    else:
        some_gdf.plot(column=field_col, ax=ax, legend=True, **kwargs)
    if name_col is not None:
        for idx, row in some_gdf.iterrows():
            ax.annotate(text=row[name_col], xy=row['coords'],
                         horizontalalignment='center')

In [ ]:
def calc_overall_centroid(some_gdf):
    """
    Use the 'coords' and 'area' columns to estimate an overall centroid
    """
    sum_x = 0.0
    sum_y = 0.0
    sum_area = 0.0
    for idx, row in some_gdf.iterrows():
        coord_x, coord_y = row['coords']
        sum_x += coord_x * row['area']
        sum_y += coord_y * row['area']
        sum_area += row['area']
    centroid_x = sum_x / sum_area
    centroid_y = sum_y / sum_area
    return centroid_x, centroid_y

In [ ]:
add_area_and_label_coords(gdf)
centroid_x, centroid_y = calc_overall_centroid(gdf)
print(centroid_x, centroid_y)

In [ ]:
fig, axes = plt.subplots(1,2)
plot_with_labels(gdf, field_col='area', name_col='NAME', ax=axes[0])
plot_with_labels(gdf, ax=axes[1])

In [ ]:
gdf.crs


# Orthographic projection centered on our centroid
See [Orthographic projection with pyproj for penguin tracking in Antarctica](https://towardsdatascience.com/orthographic-projection-with-pyproj-for-penguin-tracking-in-antarctica-18cd2bf2d570) for the trick.

In [ ]:
lat = centroid_y
lon = centroid_x
ortho = CRS.from_proj4("+proj=ortho +lat_0={} +lon_0={} +x_0=0 +y_0=0 +a=6370997 +b=6370997 +units=m +no_defs".format(lat, lon))

In [ ]:
ortho_gdf = gdf.to_crs(ortho)
ortho_gdf.crs

In [ ]:
utm_proj = CRS.from_proj4("+proj=utm +zone=10 +north")
print(utm_proj)

In [ ]:
utm_gdf = gdf.to_crs(utm_proj)
utm_gdf.crs

In [ ]:
utm_gdf.plot()

In [ ]:
fig, axes = plt.subplots(1,2)
plot_with_labels(gdf, ax=axes[0])
plot_with_labels(ortho_gdf, ax=axes[1])

In [ ]:
new_gdf = gdf.to_crs("EPSG:32633")

In [ ]:
new_gdf.crs

In [ ]:
fig, axes = plt.subplots(1,2)
plot_with_labels(gdf, ax=axes[0])
plot_with_labels(new_gdf, ax=axes[1])

In [ ]:
gdf = gpd.read_file('/home/welling/geo/USA/all/tl_2021_us_county.shp')
display(gdf.columns)
gdf = gdf[gdf.STATEFP=='42']  # select PA
add_area_and_label_coords(gdf)

In [ ]:
def build_fips_string(row):
    return f"{row['STATEFP']}{row['COUNTYFP']}"
gdf['fips_string'] = gdf.apply(build_fips_string, axis=1)
gdf.head()

In [ ]:
fig, axes = plt.subplots(2,1)
plot_with_labels(gdf, name_col='NAME', ax=axes[0])
plot_with_labels(gdf, name_col='fips_string', ax=axes[1])

In [ ]:
geolocator = Nominatim(user_agent="ms_das_vis")
location = geolocator.geocode("425 South Craig St., Pittsburgh")
print(f'address: {location.address}')
print(f'point: {location.point}')
#print(f'raw: {location.raw}')
print(f'altitude: {location.altitude}')
print(f'latitude: {location.latitude}')
print(f'longitude: {location.longitude}')

In [ ]:
geolocator.reverse((40.44501531264918, -79.94863300565322), exactly_one=False)  # (latitude, longitude)